# RAG-Based Question Answering System
## Using LangChain, HuggingFace, and FAISS

This notebook demonstrates a Retrieval-Augmented Generation (RAG) system that answers questions based on PDF documents.

In [ ]:
# --------------------------
# Step 1: Import Libraries
# --------------------------
import os
from getpass import getpass
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
# --------------------------
# Step 2: Set Hugging Face Token (Securely)
# --------------------------
# Option 1: Use environment variable (recommended for production)
# Set this in your terminal before running: export HUGGINGFACEHUB_API_TOKEN="your_token_here"

# Option 2: Input token securely (recommended for notebooks)
if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your HuggingFace API token: ")
else:
    print("Using HuggingFace token from environment variable")

In [ ]:
# --------------------------
# Step 3: Configuration Parameters
# --------------------------
CONFIG = {
    "pdf_path": "A_Brief_Introduction_To_AI.pdf",
    "chunk_size": 500,
    "chunk_overlap": 100,
    "embedding_model": "all-MiniLM-L6-v2",
    "llm_model": "HuggingFaceH4/zephyr-7b-beta",
    "temperature": 0.5,
    "max_tokens": 512,
    "retriever_k": 4  # Number of documents to retrieve
}

In [ ]:
# --------------------------
# Step 4: Load and Chunk the PDF
# --------------------------
try:
    print(f"Loading PDF: {CONFIG['pdf_path']}")
    loader = PyPDFLoader(CONFIG['pdf_path'])
    pages = loader.load()
    print(f"✓ Loaded {len(pages)} pages")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CONFIG['chunk_size'],
        chunk_overlap=CONFIG['chunk_overlap']
    )
    documents = text_splitter.split_documents(pages)
    print(f"✓ Split into {len(documents)} chunks")
    
except FileNotFoundError:
    print(f"Error: PDF file '{CONFIG['pdf_path']}' not found.")
    print("Please ensure the PDF file is in the same directory as this notebook.")
except Exception as e:
    print(f"Error loading PDF: {str(e)}")

In [ ]:
# --------------------------
# Step 5: Generate Embeddings and Create Vector Store
# --------------------------
try:
    print(f"Creating embeddings using {CONFIG['embedding_model']}...")
    embedding_model = HuggingFaceEmbeddings(model_name=CONFIG['embedding_model'])
    
    print("Building FAISS vector store...")
    db = FAISS.from_documents(documents, embedding_model)
    print("✓ Vector store created successfully")
    
    # Optional: Save the vector store for future use
    # db.save_local("faiss_index")
    # print("✓ Vector store saved to disk")
    
except Exception as e:
    print(f"Error creating vector store: {str(e)}")

In [ ]:
# --------------------------
# Step 6: Set Up LLM and RAG Chain
# --------------------------
try:
    print(f"Initializing LLM: {CONFIG['llm_model']}...")
    llm = HuggingFaceEndpoint(
        repo_id=CONFIG['llm_model'],
        temperature=CONFIG['temperature'],
        max_new_tokens=CONFIG['max_tokens']
    )
    
    print("Creating RAG chain...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=db.as_retriever(search_kwargs={"k": CONFIG['retriever_k']}),
        return_source_documents=True
    )
    print("✓ RAG chain ready")
    
except Exception as e:
    print(f"Error setting up RAG chain: {str(e)}")

In [ ]:
# --------------------------
# Step 7: Question Answering Helper Function
# --------------------------
def ask_question(query, show_sources=True, max_source_length=500):
    """
    Ask a question and get an answer based on the PDF content.
    
    Args:
        query (str): The question to ask
        show_sources (bool): Whether to display source documents
        max_source_length (int): Maximum characters to show from each source
    """
    try:
        result = qa_chain.invoke({"query": query})
        
        print("="*80)
        print(f"Question: {query}")
        print("="*80)
        print(f"\nAnswer:\n{result['result']}\n")
        
        if show_sources and result.get("source_documents"):
            print("\n" + "="*80)
            print("Source Documents:")
            print("="*80)
            for i, doc in enumerate(result["source_documents"], 1):
                print(f"\n--- Source {i} ---")
                content = doc.page_content[:max_source_length]
                if len(doc.page_content) > max_source_length:
                    content += "..."
                print(content)
                
                # Show metadata if available
                if doc.metadata:
                    print(f"\nMetadata: {doc.metadata}")
                print("-" * 80)
                
    except Exception as e:
        print(f"Error processing question: {str(e)}")

In [ ]:
# --------------------------
# Step 8: Example Questions
# --------------------------
# Example 1: Basic question
ask_question("What is Artificial Intelligence?")

In [ ]:
# Example 2: More specific question
ask_question("What are the subfields of AI mentioned in the document?")

In [ ]:
# Example 3: Question without showing sources
ask_question("How does machine learning work?", show_sources=False)

In [ ]:
# --------------------------
# Interactive Question Answering
# --------------------------
# Uncomment the following lines to enable interactive mode

# while True:
#     user_query = input("\nEnter your question (or 'quit' to exit): ")
#     if user_query.lower() in ['quit', 'exit', 'q']:
#         print("Goodbye!")
#         break
#     ask_question(user_query)